# Import Data

In [0]:
# connect so s3 bucket
# get credentials
import os

ACCESS_KEY = os.getenv("AWS_ACCESS_KEY")
SECRET_KEY = os.getenv("AWS_SECRET_KEY")
ENCODED_SECRET_KEY = SECRET_KEY.replace("/", "%2F")
AWS_BUCKET_NAME = "aida-project"
MOUNT_NAME = "data"

# mount data
try:
  dbutils.fs.mount("s3a://%s:%s@%s" % (ACCESS_KEY, ENCODED_SECRET_KEY, AWS_BUCKET_NAME), "/mnt/%s" % MOUNT_NAME)
except:
  display(dbutils.fs.ls("/mnt/%s" % MOUNT_NAME))

path,name,size
dbfs:/mnt/data/,data/,0
dbfs:/mnt/data/TSV/,TSV/,0
dbfs:/mnt/data/niy/,niy/,0
dbfs:/mnt/data/team-remote/,team-remote/,0
dbfs:/mnt/data/team_remote/,team_remote/,0


In [0]:
display(dbutils.fs.ls("/mnt/%s/TSV" % MOUNT_NAME))

path,name,size
dbfs:/mnt/data/TSV/name.basics.tsv,name.basics.tsv,579976550
dbfs:/mnt/data/TSV/title.akas.tsv,title.akas.tsv,969441812
dbfs:/mnt/data/TSV/title.basics.tsv,title.basics.tsv,537519832
dbfs:/mnt/data/TSV/title.principals.tsv,title.principals.tsv,1622240736
dbfs:/mnt/data/TSV/title.ratings.tsv,title.ratings.tsv,16907124


In [0]:
df_names = spark.read.load("dbfs:/mnt/data/TSV/name.basics.tsv",
                           format="csv", sep="\t", inferSchema="true", header="true")
df_akas = spark.read.load("dbfs:/mnt/data/TSV/title.akas.tsv",
                           format="csv", sep="\t", inferSchema="true", header="true")
df_basics = spark.read.load("dbfs:/mnt/data/TSV/title.basics.tsv",
                           format="csv", sep="\t", inferSchema="true", header="true")
df_principals = spark.read.load("dbfs:/mnt/data/TSV/title.principals.tsv",
                           format="csv", sep="\t", inferSchema="true", header="true")
df_ratings = spark.read.load("dbfs:/mnt/data/TSV/title.ratings.tsv",
                           format="csv", sep="\t", inferSchema="true", header="true")

In [0]:
list_dfs = [df_names, df_akas, df_basics, df_principals, df_ratings]

for df in list_dfs:
  df.printSchema()

root
-- nconst: string (nullable = true)
-- primaryName: string (nullable = true)
-- birthYear: string (nullable = true)
-- deathYear: string (nullable = true)
-- primaryProfession: string (nullable = true)
-- knownForTitles: string (nullable = true)

root
-- titleId: string (nullable = true)
-- ordering: integer (nullable = true)
-- title: string (nullable = true)
-- region: string (nullable = true)
-- language: string (nullable = true)
-- types: string (nullable = true)
-- attributes: string (nullable = true)
-- isOriginalTitle: string (nullable = true)

root
-- tconst: string (nullable = true)
-- titleType: string (nullable = true)
-- primaryTitle: string (nullable = true)
-- originalTitle: string (nullable = true)
-- isAdult: integer (nullable = true)
-- startYear: string (nullable = true)
-- endYear: string (nullable = true)
-- runtimeMinutes: string (nullable = true)
-- genres: string (nullable = true)

root
-- tconst: string (nullable = true)
-- ordering: integer (nullable = true)
-- nconst: string (nullable = true)
-- category: string (nullable = true)
-- job: string (nullable = true)
-- characters: string (nullable = true)

root
-- tconst: string (nullable = true)
-- averageRating: double (nullable = true)
-- numVotes: integer (nullable = true)

# Feature Selection & Data Cleaning

In [0]:
from pyspark.sql.functions import mean as _mean, \
                                  min as _min, \
                                  max as _max, \
                                  count as _count, \
                                  stddev as _stddev, \
                                  countDistinct, col, isnan, split, array_distinct, explode

## Sampling

In [0]:
# until final model evaluation use sample

random_state = 42
sample_size = 1.0

train_size = 0.8
test_size = 1 - train_size

def sample(dataframe):
  df_ids = dataframe.select('tconst', 'averageRating').sample(sample_size, random_state)
  return df_ids

## Table: Ratings

In [0]:
def votes(dataframe):
  df_ids = dataframe.join(df_ratings, ['tconst', 'averageRating'])
  return df_ids

## Table: Principals

In [0]:
def principals(dataframe):
  df_a = dataframe.join(df_principals.drop('job','characters'), on='tconst')\
      .groupBy('tconst', 'averageRating', 'numVotes')\
      .agg(_max('ordering').alias('principal_counts'), countDistinct('category').alias('distinct_count_categories'))
  
  df_b = dataframe.join(df_principals, on='tconst').select('tconst', 'category').groupBy('tconst').pivot('category').count().fillna(0)
  
  df_ids = df_a.join(df_b, on='tconst')
  
  return df_ids

## Table: AKAS

In [0]:
from pyspark.sql.functions import split, explode, array_distinct
#df_akas_sample = df_akas.sample(False, 0.000001, 42)
df_akas_sample = df_akas

def akas(dataframe):
  droplist = ['title', 'language', 'types', 'attributes', 'isOriginalTitle']
  df_akas_feature = df_akas_sample.drop(*droplist)
  column = 'region'
  df_akas_feature = df_akas_feature.where(fr'{column} != "\\N"')
  df_akas_pivot = df_akas_feature.groupBy('titleId').pivot('region').count().fillna(0)
  df_akas_pivot = df_akas_pivot.withColumnRenamed('titleId', 'tconst')
  df_ids = dataframe.join(df_akas_pivot, on='tconst')
  return df_ids

## Table: Basics

In [0]:
def basics(dataframe):
  
  droplist = ['primaryTitle', 'originalTitle', 'endYear']
  df_basics_sample_fs = df_basics.drop(*droplist)
  
  df_basics_sample_pivot = df_basics_sample_fs.select('tconst', 'genres', explode(array_distinct(split('genres', ',')))).groupBy('tconst').pivot('col').count().fillna(0)
  df_basics_sample_pivot = df_basics_sample_pivot.drop(r'\N')
                                                                              
  df_ids = dataframe.join(df_basics_sample_pivot, on='tconst')\
     
  return df_ids

# Train-Test Split

In [0]:
df_ids = sample(df_ratings)
df_ids = votes(df_ids)
df_ids = principals(df_ids)
df_ids = akas(df_ids)
df_ids = basics(df_ids)
df_ids.printSchema()

root
-- tconst: string (nullable = true)
-- averageRating: double (nullable = true)
-- numVotes: integer (nullable = true)
-- principal_counts: integer (nullable = true)
-- distinct_count_categories: long (nullable = false)
-- actor: long (nullable = true)
-- actress: long (nullable = true)
-- archive_footage: long (nullable = true)
-- archive_sound: long (nullable = true)
-- cinematographer: long (nullable = true)
-- composer: long (nullable = true)
-- director: long (nullable = true)
-- editor: long (nullable = true)
-- producer: long (nullable = true)
-- production_designer: long (nullable = true)
-- self: long (nullable = true)
-- writer: long (nullable = true)
-- AD: long (nullable = true)
-- AE: long (nullable = true)
-- AF: long (nullable = true)
-- AG: long (nullable = true)
-- AI: long (nullable = true)
-- AL: long (nullable = true)
-- AM: long (nullable = true)
-- AN: long (nullable = true)
-- AO: long (nullable = true)
-- AQ: long (nullable = true)
-- AR: long (nullable = true)
-- AS: long (nullable = true)
-- AT: long (nullable = true)
-- AU: long (nullable = true)
-- AW: long (nullable = true)
-- AZ: long (nullable = true)
-- BA: long (nullable = true)
-- BB: long (nullable = true)
-- BD: long (nullable = true)
-- BE: long (nullable = true)
-- BF: long (nullable = true)
-- BG: long (nullable = true)
-- BH: long (nullable = true)
-- BI: long (nullable = true)
-- BJ: long (nullable = true)
-- BM: long (nullable = true)
-- BN: long (nullable = true)
-- BO: long (nullable = true)
-- BR: long (nullable = true)
-- BS: long (nullable = true)
-- BT: long (nullable = true)
-- BUMM: long (nullable = true)
-- BW: long (nullable = true)
-- BY: long (nullable = true)
-- BZ: long (nullable = true)
-- CA: long (nullable = true)
-- CD: long (nullable = true)
-- CF: long (nullable = true)
-- CG: long (nullable = true)
-- CH: long (nullable = true)
-- CI: long (nullable = true)
-- CK: long (nullable = true)
-- CL: long (nullable = true)
-- CM: long (nullable = true)
-- CN: long (nullable = true)
-- CO: long (nullable = true)
-- CR: long (nullable = true)
-- CSHH: long (nullable = true)
-- CSXX: long (nullable = true)
-- CU: long (nullable = true)
-- CV: long (nullable = true)
-- CY: long (nullable = true)
-- CZ: long (nullable = true)
-- DDDE: long (nullable = true)
-- DE: long (nullable = true)
-- DJ: long (nullable = true)
-- DK: long (nullable = true)
-- DM: long (nullable = true)
-- DO: long (nullable = true)
-- DZ: long (nullable = true)
-- EC: long (nullable = true)
-- EE: long (nullable = true)
-- EG: long (nullable = true)
-- EH: long (nullable = true)
-- ER: long (nullable = true)
-- ES: long (nullable = true)
-- ET: long (nullable = true)
-- FI: long (nullable = true)
-- FJ: long (nullable = true)
-- FO: long (nullable = true)
-- FR: long (nullable = true)
-- GA: long (nullable = true)
-- GB: long (nullable = true)
-- GD: long (nullable = true)
-- GE: long (nullable = true)
-- GF: long (nullable = true)
-- GH: long (nullable = true)
-- GI: long (nullable = true)
-- GL: long (nullable = true)
-- GM: long (nullable = true)
-- GN: long (nullable = true)
-- GP: long (nullable = true)
-- GQ: long (nullable = true)
-- GR: long (nullable = true)
-- GT: long (nullable = true)
-- GU: long (nullable = true)
-- GW: long (nullable = true)
-- GY: long (nullable = true)
-- HK: long (nullable = true)
-- HN: long (nullable = true)
-- HR: long (nullable = true)
-- HT: long (nullable = true)
-- HU: long (nullable = true)
-- ID: long (nullable = true)
-- IE: long (nullable = true)
-- IL: long (nullable = true)
-- IM: long (nullable = true)
-- IN: long (nullable = true)
-- IQ: long (nullable = true)
-- IR: long (nullable = true)
-- IS: long (nullable = true)
-- IT: long (nullable = true)
-- JE: long (nullable = true)
-- JM: long (nullable = true)
-- JO: long (nullable = true)
-- JP: long (nullable = true)
-- KE: long (nullable = true)
-- KG: long (nullable = true)
-- KH: long (nullable = true)
-- KI: long (nullable = true)
-- KM: long (nulla

In [0]:
df_train, df_test = df_ids.randomSplit([train_size, test_size], random_state)

In [0]:
df_train.head()

Out[17]: Row(tconst='tt0000015', averageRating=6.1, numVotes=742, principal_counts=1, distinct_count_categories=1, actor=0, actress=0, archive_footage=0, archive_sound=0, cinematographer=0, composer=0, director=1, editor=0, producer=0, production_designer=0, self=0, writer=0, AD=0, AE=0, AF=0, AG=0, AI=0, AL=0, AM=0, AN=0, AO=0, AQ=0, AR=0, AS=0, AT=0, AU=0, AW=0, AZ=0, BA=0, BB=0, BD=0, BE=0, BF=0, BG=0, BH=0, BI=0, BJ=0, BM=0, BN=0, BO=0, BR=0, BS=0, BT=0, BUMM=0, BW=0, BY=0, BZ=0, CA=0, CD=0, CF=0, CG=0, CH=0, CI=0, CK=0, CL=0, CM=0, CN=0, CO=0, CR=0, CSHH=0, CSXX=0, CU=0, CV=0, CY=0, CZ=0, DDDE=0, DE=0, DJ=0, DK=0, DM=0, DO=0, DZ=0, EC=0, EE=0, EG=0, EH=0, ER=0, ES=0, ET=0, FI=0, FJ=0, FO=0, FR=1, GA=0, GB=0, GD=0, GE=0, GF=0, GH=0, GI=0, GL=0, GM=0, GN=0, GP=0, GQ=0, GR=0, GT=0, GU=0, GW=0, GY=0, HK=0, HN=0, HR=0, HT=0, HU=1, ID=0, IE=0, IL=0, IM=0, IN=0, IQ=0, IR=0, IS=0, IT=0, JE=0, JM=0, JO=0, JP=0, KE=0, KG=0, KH=0, KI=0, KM=0, KN=0, KP=0, KR=0, KW=0, KY=0, KZ=0, LA=0, LB=0, LC=0, LI=0, LK=0, LR=0, LS=0, LT=0, LU=0, LV=0, LY=0, MA=0, MC=0, MD=0, ME=0, MG=0, MH=0, MK=0, ML=0, MM=0, MN=0, MO=0, MP=0, MQ=0, MR=0, MS=0, MT=0, MU=0, MV=0, MW=0, MX=0, MY=0, MZ=0, NA=0, NC=0, NE=0, NG=0, NI=0, NL=0, NO=0, NP=0, NR=0, NU=0, NZ=0, OM=0, PA=0, PE=0, PF=0, PG=0, PH=0, PK=0, PL=0, PR=0, PS=0, PT=0, PW=0, PY=0, QA=0, RE=0, RO=0, RS=0, RU=1, RW=0, SA=0, SB=0, SC=0, SD=0, SE=0, SG=0, SH=0, SI=0, SK=0, SL=0, SM=0, SN=0, SO=0, SR=0, ST=0, SUHH=0, SV=0, SY=0, SZ=0, TD=0, TG=0, TH=0, TJ=0, TL=0, TM=0, TN=0, TO=0, TR=0, TT=0, TV=0, TW=0, TZ=0, UA=0, UG=0, US=1, UY=0, UZ=0, VA=0, VC=0, VDVN=0, VE=0, VG=0, VI=0, VN=0, VU=0, WF=0, WS=0, XAS=0, XAU=0, XEU=0, XKO=0, XKV=0, XNA=0, XPI=0, XSA=0, XSI=0, XWG=0, XWW=0, XYU=0, YE=0, YUCS=0, ZA=0, ZM=0, ZRCD=0, ZW=0, Action=0, Adult=0, Adventure=0, Animation=1, Biography=0, Comedy=0, Crime=0, Documentary=0, Drama=0, Family=0, Fantasy=0, Film-Noir=0, Game-Show=0, History=0, Horror=0, Music=0, Musical=0, Mystery=0, News=0, Reality-TV=0, Romance=0, Sci-Fi=0, Short=1, Sport=0, Talk-Show=0, Thriller=0, War=0, Western=0)

In [0]:
df_train.drop('tconst') \
  .repartition(1) \
  .write.option("header", "false") \
  .save(f'/mnt/data/niy/{label}', format='csv')

Out[14]: ['tconst',
 'averageRating',
 'numVotes',
 'principal_counts',
 'distinct_count_categories',
 'actor',
 'actress',
 'archive_footage',
 'archive_sound',
 'cinematographer',
 'composer',
 'director',
 'editor',
 'producer',
 'production_designer',
 'self',
 'writer',
 'AD',
 'AE',
 'AF',
 'AG',
 'AI',
 'AL',
 'AM',
 'AN',
 'AO',
 'AQ',
 'AR',
 'AS',
 'AT',
 'AU',
 'AW',
 'AZ',
 'BA',
 'BB',
 'BD',
 'BE',
 'BF',
 'BG',
 'BH',
 'BI',
 'BJ',
 'BM',
 'BN',
 'BO',
 'BR',
 'BS',
 'BT',
 'BUMM',
 'BW',
 'BY',
 'BZ',
 'CA',
 'CD',
 'CF',
 'CG',
 'CH',
 'CI',
 'CK',
 'CL',
 'CM',
 'CN',
 'CO',
 'CR',
 'CSHH',
 'CSXX',
 'CU',
 'CV',
 'CY',
 'CZ',
 'DDDE',
 'DE',
 'DJ',
 'DK',
 'DM',
 'DO',
 'DZ',
 'EC',
 'EE',
 'EG',
 'EH',
 'ER',
 'ES',
 'ET',
 'FI',
 'FJ',
 'FO',
 'FR',
 'GA',
 'GB',
 'GD',
 'GE',
 'GF',
 'GH',
 'GI',
 'GL',
 'GM',
 'GN',
 'GP',
 'GQ',
 'GR',
 'GT',
 'GU',
 'GW',
 'GY',
 'HK',
 'HN',
 'HR',
 'HT',
 'HU',
 'ID',
 'IE',
 'IL',
 'IM',
 'IN',
 'IQ',
 'IR',
 'IS',
 'IT',
 'JE',
 'JM',
 'JO',
 'JP',
 'KE',
 'KG',
 'KH',
 'KI',
 'KM',
 'KN',
 'KP',
 'KR',
 'KW',
 'KY',
 'KZ',
 'LA',
 'LB',
 'LC',
 'LI',
 'LK',
 'LR',
 'LS',
 'LT',
 'LU',
 'LV',
 'LY',
 'MA',
 'MC',
 'MD',
 'ME',
 'MG',
 'MH',
 'MK',
 'ML',
 'MM',
 'MN',
 'MO',
 'MP',
 'MQ',
 'MR',
 'MS',
 'MT',
 'MU',
 'MV',
 'MW',
 'MX',
 'MY',
 'MZ',
 'NA',
 'NC',
 'NE',
 'NG',
 'NI',
 'NL',
 'NO',
 'NP',
 'NR',
 'NU',
 'NZ',
 'OM',
 'PA',
 'PE',
 'PF',
 'PG',
 'PH',
 'PK',
 'PL',
 'PR',
 'PS',
 'PT',
 'PW',
 'PY',
 'QA',
 'RE',
 'RO',
 'RS',
 'RU',
 'RW',
 'SA',
 'SB',
 'SC',
 'SD',
 'SE',
 'SG',
 'SH',
 'SI',
 'SK',
 'SL',
 'SM',
 'SN',
 'SO',
 'SR',
 'ST',
 'SUHH',
 'SV',
 'SY',
 'SZ',
 'TD',
 'TG',
 'TH',
 'TJ',
 'TL',
 'TM',
 'TN',
 'TO',
 'TR',
 'TT',
 'TV',
 'TW',
 'TZ',
 'UA',
 'UG',
 'US',
 'UY',
 'UZ',
 'VA',
 'VC',
 'VDVN',
 'VE',
 'VG',
 'VI',
 'VN',
 'VU',
 'WF',
 'WS',
 'XAS',
 'XAU',
 'XEU',
 'XKO',
 'XKV',
 'XNA',
 'XPI',
 'XSA',
 'XSI',
 'XWG',
 'XWW',
 'XYU',
 'YE',
 'YUCS',
 'ZA',
 'ZM',
 'ZRCD',
 'ZW',
 'Action',
 'Adult',
 'Adventure',
 'Animation',
 'Biography',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Film-Noir',
 'Game-Show',
 'History',
 'Horror',
 'Music',
 'Musical',
 'Mystery',
 'News',
 'Reality-TV',
 'Romance',
 'Sci-Fi',
 'Short',
 'Sport',
 'Talk-Show',
 'Thriller',
 'War',
 'Western']

# Write to S3

In [0]:
def save_to_s3(dataframe, label):
    dataframe.drop('tconst') \
      .repartition(1) \
      .write.option("header", "false") \
      .save(f'/mnt/data/niy/{label}', format='csv')
    return "Successfully saved to s3"

In [0]:
def partition_to_s3(dataframe, label):
  dataframe.drop('tconst') \
    .write.option("header", "false") \
    .save(f'/mnt/data/niy/{label}', format='csv')
  return "Successfully saved to s3"

In [0]:
def copyMerge (src_dir, dst_file, overwrite=False, deleteSource=False, debug=False):
    
    # this function has been migrated to https://github.com/Tagar/abalon Python package
    
    hadoop = sc._jvm.org.apache.hadoop
    conf = hadoop.conf.Configuration()
    fs = hadoop.fs.FileSystem.get(conf)

    # check files that will be merged
    files = []
    for f in fs.listStatus(hadoop.fs.Path(src_dir)):
        if f.isFile():
            files.append(f.getPath())
    if not files:
        raise ValueError("Source directory {} is empty".format(src_dir))
    files.sort(key=lambda f: str(f))

    # dst_permission = hadoop.fs.permission.FsPermission.valueOf(permission)      # , permission='-rw-r-----'
    out_stream = fs.create(hadoop.fs.Path(dst_file), overwrite)

    try:
        # loop over files in alphabetical order and append them one by one to the target file
        for file in files:
            if debug: 
                print("Appending file {} into {}".format(file, dst_file))

            in_stream = fs.open(file)   # InputStream object
            try:
                hadoop.io.IOUtils.copyBytes(in_stream, out_stream, conf, False)     # False means don't close out_stream
            finally:
                in_stream.close()
    finally:
        out_stream.close()

    if deleteSource:
        fs.delete(hadoop.fs.Path(src_dir), True)    # True=recursive
        if debug:
            print("Source directory {} removed.".format(src_dir))

In [0]:
if sample_size < 1:
  ############### WRITE TRAINING DATA #################
  try:
    save_to_s3(df_train, 'train')
  except:
    dbutils.fs.rm('mnt/data/niy/train', True)
    save_to_s3(df_train, 'train')

  file = dbutils.fs.ls('mnt/data/niy/train')[-1].path
  dbutils.fs.cp(file, '/mnt/data/niy/train.csv')
  dbutils.fs.rm(file)
  print('Successfully saved training data')
  
  ############### WRITE TESTING DATA #################
  try:
    save_to_s3(df_test, 'test')
  except:
    dbutils.fs.rm('mnt/data/niy/test', True)
    save_to_s3(df_test, 'test')

  file = dbutils.fs.ls('mnt/data/niy/test')[-1].path
  dbutils.fs.cp(file, '/mnt/data/niy/test.csv')
  dbutils.fs.rm(file)
  print('Successfully saved testing data')

else:
  try:
    dbutils.fs.rm('mnt/data/niy/merged', True)
    print('Successfully deleted data')
  except:
    print('Data space does not exist yet')
  
  ############### WRITE TRAINING DATA #################  
  try:
    partition_to_s3(df_train, 'partitioned_train')
  except:
    dbutils.fs.rm('mnt/data/niy/partitioned_train', True)
    partition_to_s3(df_train, 'partitioned_train')
    
  # delete the empty files  
  for file in dbutils.fs.ls('mnt/data/niy/partitioned_train')[:3]:
    dbutils.fs.rm(file.path)
    print(f'Successfully deleted {file.path}')
  
  copyMerge('/mnt/data/niy/partitioned_train', '/mnt/data/niy/merged/train.csv', debug=True, overwrite=True, deleteSource=False)
  
  ############### WRITE TESTING DATA #################
  try:
    partition_to_s3(df_test, 'partitioned_test')
  except:
    dbutils.fs.rm('mnt/data/niy/partitioned_test', True)
    partition_to_s3(df_test, 'partitioned_train')
    
  # delete the empty files  
  for file in dbutils.fs.ls('mnt/data/niy/partitioned_test')[:3]:
    dbutils.fs.rm(file.path)
    print(f'Successfully deleted {file.path}')
  
  copyMerge('/mnt/data/niy/partitioned_test', '/mnt/data/niy/merged/test.csv', debug=True, overwrite=True, deleteSource=False)  

Successfully deleted data
Successfully deleted dbfs:/mnt/data/niy/partitioned_train/_SUCCESS
Successfully deleted dbfs:/mnt/data/niy/partitioned_train/_committed_3660031154883285517
Successfully deleted dbfs:/mnt/data/niy/partitioned_train/_started_3660031154883285517
Appending file dbfs:/mnt/data/niy/partitioned_train/part-00000-tid-3660031154883285517-ef87f4ef-3d83-4198-aa5f-c3dfa5fc83ba-891-1-c000.csv into /mnt/data/niy/merged/train.csv
Appending file dbfs:/mnt/data/niy/partitioned_train/part-00001-tid-3660031154883285517-ef87f4ef-3d83-4198-aa5f-c3dfa5fc83ba-892-1-c000.csv into /mnt/data/niy/merged/train.csv
Appending file dbfs:/mnt/data/niy/partitioned_train/part-00002-tid-3660031154883285517-ef87f4ef-3d83-4198-aa5f-c3dfa5fc83ba-893-1-c000.csv into /mnt/data/niy/merged/train.csv
Appending file dbfs:/mnt/data/niy/partitioned_train/part-00003-tid-3660031154883285517-ef87f4ef-3d83-4198-aa5f-c3dfa5fc83ba-894-1-c000.csv into /mnt/data/niy/merged/train.csv
Appending file dbfs:/mnt/data/niy/partitioned_train/part-00004-tid-3660031154883285517-ef87f4ef-3d83-4198-aa5f-c3dfa5fc83ba-895-1-c000.csv into /mnt/data/niy/merged/train.csv
Appending file dbfs:/mnt/data/niy/partitioned_train/part-00005-tid-3660031154883285517-ef87f4ef-3d83-4198-aa5f-c3dfa5fc83ba-896-1-c000.csv into /mnt/data/niy/merged/train.csv
Appending file dbfs:/mnt/data/niy/partitioned_train/part-00006-tid-3660031154883285517-ef87f4ef-3d83-4198-aa5f-c3dfa5fc83ba-897-1-c000.csv into /mnt/data/niy/merged/train.csv
Appending file dbfs:/mnt/data/niy/partitioned_train/part-00007-tid-3660031154883285517-ef87f4ef-3d83-4198-aa5f-c3dfa5fc83ba-898-1-c000.csv into /mnt/data/niy/merged/train.csv
Appending file dbfs:/mnt/data/niy/partitioned_train/part-00008-tid-3660031154883285517-ef87f4ef-3d83-4198-aa5f-c3dfa5fc83ba-899-1-c000.csv into /mnt/data/niy/merged/train.csv
Appending file dbfs:/mnt/data/niy/partitioned_train/part-00009-tid-3660031154883285517-ef87f4ef-3d83-4198-aa5f-c3dfa5fc83ba-900-1-c000.csv into /mnt/data/niy/merged/train.csv
Appending file dbfs:/mnt/data/niy/partitioned_train/part-00010-tid-3660031154883285517-ef87f4ef-3d83-4198-aa5f-c3dfa5fc83ba-901-1-c000.csv into /mnt/data/niy/merged/train.csv
Appending file dbfs:/mnt/data/niy/partitioned_train/part-00011-tid-3660031154883285517-ef87f4ef-3d83-4198-aa5f-c3dfa5fc83ba-902-1-c000.csv into /mnt/data/niy/merged/train.csv
Appending file dbfs:/mnt/data/niy/partitioned_train/part-00012-tid-3660031154883285517-ef87f4ef-3d83-4198-aa5f-c3dfa5fc83ba-903-1-c000.csv into /mnt/data/niy/merged/train.csv
Appending file dbfs:/mnt/data/niy/partitioned_train/part-00013-tid-3660031154883285517-ef87f4ef-3d83-4198-aa5f-c3dfa5fc83ba-904-1-c000.csv into /mnt/data/niy/merged/train.csv
Appending file dbfs:/mnt/data/niy/partitioned_train/part-00014-tid-3660031154883285517-ef87f4ef-3d83-4198-aa5f-c3dfa5fc83ba-905-1-c000.csv into /mnt/data/niy/merged/train.csv
Appending file dbfs:/mnt/data/niy/partitioned_train/part-00015-tid-3660031154883285517-ef87f4ef-3d83-4198-aa5f-c3dfa5fc83ba-906-1-c000.csv into /mnt/data/niy/merged/train.csv
Appending file dbfs:/mnt/data/niy/partitioned_train/part-00016-tid-3660031154883285517-ef87f4ef-3d83-4198-aa5f-c3dfa5fc83ba-907-1-c000.csv into /mnt/data/niy/merged/train.csv
Successfully deleted dbfs:/mnt/data/niy/partitioned_test/_SUCCESS
Successfully deleted dbfs:/mnt/data/niy/partitioned_test/_committed_4153331303712694732
Successfully deleted dbfs:/mnt/data/niy/partitioned_test/_started_4153331303712694732
Appending file dbfs:/mnt/data/niy/partitioned_test/part-00000-tid-4153331303712694732-ed0a93d1-12c1-4241-a736-2d133dc38feb-1045-1-c000.csv into /mnt/data/niy/merged/test.csv
Appending file dbfs:/mnt/data/niy/partitioned_test/part-00001-tid-4153331303712694732-ed0a93d1-12c1-4241-a736-2d133dc38feb-1046-1-c000.csv into /mnt/data/niy/merged/test.csv
Appending file dbfs:/mnt/data/niy/partitioned_test/part-00002-tid-4153331303712694732-ed0a93d1-12c1-4241-a736-2d133dc38feb-1047-1-c000.csv into /mnt/data/niy/merged/tes